# 导包和pyspark环境设置

In [1]:
import findspark
findspark.init()
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf
from pyspark import SQLContext
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder
from pyspark.sql.types import IntegerType
sc = SparkContext("local", "first")
logFile = "file:///F:/FBDP/实验/实验四/logfile1.txt"  
logData = sc.textFile(logFile).cache()
spark = SparkSession.builder.config("spark.driver.memory", "16g").getOrCreate()

# 统计所有用户所在公司类型 employer_type 的数量分布占比情况并存入csv

In [12]:
spark = SQLContext(sc)
df=spark.read.options(header='True') .csv("file:///F:/FBDP/实验/实验四/train_data.csv")
#df.printSchema() #打印数据的树形结构
#df.show()
df.createOrReplaceTempView("debit")#将DataFrame注册为SQL临时视图
sqlDF = spark.sql("SELECT * FROM debit")
#sqlDF.show()
emp_res=df.groupby('employer_type').count()
emp_res.createOrReplaceTempView("employer")
emp_res.show()
spark.sql("select employer_type,count/300000 from employer").show()
task3_1_df=spark.sql("select employer_type,count/300000 from employer").toDF("employer_type","ratio")
task3_1_df.toPandas().to_csv('task3_1_output.csv')

+--------------+------+
| employer_type| count|
+--------------+------+
|幼教与中小学校| 29995|
|      上市企业| 30038|
|      政府机构| 77446|
|    世界五百强| 16112|
|  高等教育机构| 10106|
|      普通企业|136303|
+--------------+------+

+--------------+------------------------------------------------+
| employer_type|(CAST(count AS DOUBLE) / CAST(300000 AS DOUBLE))|
+--------------+------------------------------------------------+
|幼教与中小学校|                             0.09998333333333333|
|      上市企业|                             0.10012666666666667|
|      政府机构|                             0.25815333333333335|
|    世界五百强|                            0.053706666666666666|
|  高等教育机构|                             0.03368666666666666|
|      普通企业|                              0.4543433333333333|
+--------------+------------------------------------------------+



# 统计每个用户最终须缴纳的利息金额并存入csv

In [13]:
spark.sql("select user_id,year_of_loan*monthly_payment*12-total_loan from debit").show()
task3_2_df=spark.sql("select user_id,year_of_loan*monthly_payment*12-total_loan from debit").toDF("user_id","total_money")
task3_2_df.toPandas().to_csv('task3_2_output.csv')

+-------+----------------------------------------------------------------------------------------------------------------------+
|user_id|(((CAST(year_of_loan AS DOUBLE) * CAST(monthly_payment AS DOUBLE)) * CAST(12 AS DOUBLE)) - CAST(total_loan AS DOUBLE))|
+-------+----------------------------------------------------------------------------------------------------------------------+
|      0|                                                                                                                3846.0|
|      1|                                                                                                    1840.6000000000004|
|      2|                                                                                                    10465.600000000002|
|      3|                                                                                                    1758.5200000000004|
|      4|                                                                                        

# 统计工作年限 work_year 超过 5 年的⽤户的房贷情况 censor_status 的数量分布占比情况并存入csv

In [4]:
#这有点问题
spark.sql("select user_id,censor_status,work_year from debit where work_year>'5'").show()
task3_3_df=spark.sql("select user_id,censor_status,work_year from debit where work_year>'5'")
task3_3_df.toPandas().to_csv('task3_3_output.csv')

+-------+-------------+---------+
|user_id|censor_status|work_year|
+-------+-------------+---------+
|      4|            0|  5 years|
|      6|            0|  8 years|
|     13|            1| < 1 year|
|     15|            1|  7 years|
|     20|            1|  7 years|
|     31|            0|  6 years|
|     37|            1|  5 years|
|     40|            1|  6 years|
|     44|            2| < 1 year|
|     45|            1|  6 years|
|     46|            0|  8 years|
|     49|            0|  6 years|
|     50|            1| < 1 year|
|     53|            1|  7 years|
|     54|            2| < 1 year|
|     60|            1| < 1 year|
|     63|            1|  5 years|
|     66|            2|  7 years|
|     67|            2| < 1 year|
|     68|            1|  6 years|
+-------+-------------+---------+
only showing top 20 rows

